In [ ]:
import requests
import urllib
import io
from PIL import Image
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:

def get_art_from_api(text_prompt):

    img_request = requests.post(
        "https://api.deepai.org/api/text2img",
        data={
            'text': text_prompt,
        },
        headers={'api-key': os.getenv("DEEP_AI_API_KEY")}
    ).json()
    img_url = img_request['output_url']

    img_res = urllib.request.urlopen(img_url).read()
    img = Image.open(io.BytesIO(img_res))
    
    w, h = img.size
    img = img.crop((0, 0, w/2, h/2))
    img = img.resize((300,220))
    return img
    

def put_art_on_card(card, art):
    card.paste(art, (35,65))

In [ ]:
from glob import glob
import re
def get_card_names_from_folder(folder_name):
    image_files = glob(f"../output/{folder_name}/*.jpg")
    image_files.sort()
    image_names = []
    for card in image_files:
        card_name = card.replace(f'../output/{folder_name}/','')
        card_name = card_name.replace(".jpg","")
        card_name = re.sub("[\(\[].*?[\)\]]", "", card_name)
        card_name = card_name.replace("_","")
        image_names.append(card_name)

    return list(zip(image_files,image_names))

In [ ]:
def image_to_bytes(image) -> bytes:
  imgByteArr = io.BytesIO()
  image.save(imgByteArr, format="JPEG")
  imgByteArr = imgByteArr.getvalue()
  return imgByteArr

In [ ]:
def generate_final_image(card_name, file_location):
        card_img = Image.open(file_location)
        cardImg.value = image_to_bytes(card_img)
        card_art = get_art_from_api(card_name)
        cardImg.value = image_to_bytes(card_art)
        put_art_on_card(card_img, card_art)
        sleep(2)
        cardImg.value = image_to_bytes(card_img)

In [ ]:
from time import sleep
def generate_sample(_):
    file_location, card_name = get_card_names_from_folder(folderLocationInput.value)[0]
    generate_final_image(card_name, file_location)

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, Label
loadCardsBtn = widgets.Button(description="load sample")
loadCardsBtn.on_click(generate_sample)
folderLocationInput = widgets.Text(value="My Cards")

cardImg = widgets.Image(format="jpg")

display(HBox([folderLocationInput, loadCardsBtn]))
display(cardImg)